In [2]:
# ['nigga', 'hate', 'love','ass','hell','better']
# #accuracy over all cross-validation folds: [0.6317343173431734, 0.618450184501845, 0.6140221402214022, 0.622140221402214, 0.6137370753323486]
# mean=0.62 std=0.01

# ['nigga', 'hate', 'love','ass','hell','better','bitch','fuck','dick']
# accuracy over all cross-validation folds: [0.6287822878228783, 0.6199261992619927, 0.6154981549815498, 0.6236162361623616, 0.6159527326440177]
# mean=0.62 std=0.00

# ['nigga', 'hate', 'love','ass','hell','better','bitch','fuck','dick','hey','shit','sexy','awesome']
# accuracy over all cross-validation folds: [0.6501845018450184, 0.6789667896678967, 0.6516605166051661, 0.6494464944649446, 0.6676514032496307]
# mean=0.66 std=0.01

# Having all individual words as features
# accuracy over all cross-validation folds: [0.7011070110701108, 0.6856088560885609, 0.6915129151291513, 0.7166051660516605, 0.7119645494830132]
# mean=0.70 std=0.01

#Same as above but remove all words appearing in less than 1% of all tweets, plus 2 and 3 grams
# accuracy over all cross-validation folds: [0.692250922509225, 0.6789667896678967, 0.6907749077490775, 0.7202952029520295, 0.7171344165435746]
# mean=0.70 std=0.02

#Same as above but remove all words appearing in less than 5% of all tweets, plus 2 and 3 grams
# accuracy over all cross-validation folds: [0.6464944649446495, 0.6686346863468635, 0.6575645756457564, 0.6553505535055351, 0.6506646971935007]
# mean=0.66 std=0.01

#Same as above but remove all words appearing in less than 0.5% of all tweets, plus 2 and 3 grams
# accuracy over all cross-validation folds: [0.6981549815498155, 0.6952029520295203, 0.6929889298892989, 0.7062730627306273, 0.7104874446085672]
# mean=0.70 std=0.01

#if I don`t clean the token which appears time too little, the word table of coefficients for nonhostile will be strange.(even have some japanese)
from collections import Counter
import numpy as np
import pandas as pd
import re
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

In [3]:
def load_data(datafile):
    """
    Read your data into a single pandas dataframe where
    - each row is an instance to be classified
    (this could be a tweet, user, or news article, depending on your project)
    - there is a column called `label` which stores the class label (e.g., the true
      category for this row)
    """
    df = pd.read_csv("\\Users\\11977\\elevate-osna-harassment\\osna\\data.csv")[['text', 'hostile']]
    df.columns = ['text', 'label']
    df['label'] = ['hostile' if i==1 else 'nonhostile' for i in df.label]
   # df['directed'] = ['directed' if i==1 else 'nondirected' for i in df.directed]
    return df


In [4]:
df = load_data('~/Dropbox/elevate/harassment/training_data/data.csv.gz')


In [5]:
df.label.value_counts()

hostile       3588
nonhostile    3186
Name: label, dtype: int64

In [140]:
# def make_features(df):
#     vec = DictVectorizer()
#     feature_dicts = []
#     words_to_track = ['nigga', 'hate', 'love','ass','hell','better','bitch','fuck','dick','hey','shit','sexy','awesome']
#     # will get different model for different features.
#     #words_to_track = ['you']
#     for i,row in df.iterrows():
#         features = {}
#         token_counts = Counter(re.sub('\W+', ' ', row['text'].lower()).split())
#         for w in words_to_track:
#             features[w] = token_counts[w]
#         feature_dicts.append(features)
#     X = vec.fit_transform(feature_dicts)
#     return X, vec
                
#X, vec = make_features(df)
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(min_df=0.005,ngram_range=(1,3))
X = vec.fit_transform(t for t in df['text'].values)

In [141]:
X

<6774x418 sparse matrix of type '<class 'numpy.int64'>'
	with 64446 stored elements in Compressed Sparse Row format>

In [142]:
X.shape

(6774, 418)

In [143]:
vec.vocabulary_

{'nigga': 220,
 'one': 242,
 'you': 408,
 'me': 200,
 'go': 115,
 'to': 340,
 'hell': 137,
 'ass': 26,
 'that': 309,
 'done': 78,
 'his': 142,
 'fucking': 106,
 'smh': 290,
 'that nigga': 310,
 'can': 53,
 'be': 33,
 'with': 391,
 'them': 321,
 'over': 249,
 'the': 314,
 'and': 13,
 'too': 349,
 'its': 164,
 'better': 41,
 'these': 324,
 'crazy': 61,
 'shit': 282,
 'women': 394,
 'who': 387,
 'are': 21,
 'black': 44,
 'much': 207,
 'lol': 184,
 'literally': 179,
 'bitch': 43,
 'you are': 409,
 'on': 239,
 'how': 146,
 'fat': 93,
 'cunt': 64,
 'on the': 241,
 'two': 358,
 'of': 232,
 'whole': 388,
 'of the': 234,
 'saying': 276,
 'ass nigga': 27,
 'need': 214,
 'big': 42,
 'name': 213,
 'my': 209,
 'was': 371,
 'keep': 167,
 'god': 116,
 'my nigga': 211,
 'nigga you': 225,
 'out': 247,
 'fuck': 101,
 'off': 235,
 'fuck off': 102,
 'call': 50,
 'about': 0,
 'ur': 361,
 'that shit': 311,
 'is': 156,
 'just': 166,
 'so': 291,
 'let': 173,
 'everyone': 88,
 'know': 169,
 'he': 133,
 'get': 

In [144]:
X[0,5]

0

In [145]:
X[:,3].sum()

53

In [146]:
for word, idx in vec.vocabulary_.items():
    print('%20s\t%d' % (word, X[:,idx].sum()))

               nigga	2340
                 one	249
                 you	2622
                  me	612
                  go	222
                  to	1918
                hell	67
                 ass	674
                that	1641
                done	50
                 his	321
             fucking	235
                 smh	35
          that nigga	195
                 can	418
                  be	611
                with	526
                them	204
                over	98
                 the	2716
                 and	1691
                 too	190
                 its	75
              better	101
               these	100
               crazy	45
                shit	1184
               women	54
                 who	297
                 are	515
               black	69
                much	107
                 lol	276
           literally	61
               bitch	410
             you are	115
                  on	708
                 how	263
                 fat	44
                cunt	709
   

              it was	62
              at the	55
               great	48
             cripple	35
           nigga was	55
                 yes	85
                 our	60
                cock	57
                down	86
           the nigga	70
              to say	37
               again	53
              coming	38
               to me	36
            what you	38
             you don	63
             and the	48
                 tho	60
               gotta	54
               point	45
                talk	43
            the shit	41
              around	42
             need to	54
               leave	43
                help	37
                 use	62
            from the	36
             looking	46
             fuck up	39
               trump	50
               wanna	61
              is the	52
                  am	73
               on my	35
            like you	37
             have to	50


In [147]:
vec.get_feature_names()

['about',
 'actually',
 'after',
 'again',
 'ain',
 'all',
 'all the',
 'already',
 'also',
 'always',
 'am',
 'amp',
 'an',
 'and',
 'and he',
 'and the',
 'and you',
 'another',
 'any',
 'anyone',
 'anything',
 'are',
 'are you',
 'around',
 'as',
 'as fuck',
 'ass',
 'ass nigga',
 'at',
 'at the',
 'away',
 'back',
 'bad',
 'be',
 'beat',
 'because',
 'been',
 'before',
 'being',
 'believe',
 'best',
 'better',
 'big',
 'bitch',
 'black',
 'boy',
 'bro',
 'but',
 'but you',
 'by',
 'call',
 'called',
 'calling',
 'can',
 'care',
 'cause',
 'co',
 'cock',
 'come',
 'coming',
 'could',
 'crazy',
 'cripple',
 'cum',
 'cunt',
 'damn',
 'day',
 'dead',
 'dick',
 'did',
 'didn',
 'do',
 'do you',
 'does',
 'doesn',
 'doing',
 'don',
 'don know',
 'done',
 'dont',
 'down',
 'dude',
 'dumb',
 'eat',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'exactly',
 'face',
 'faggot',
 'fat',
 'feel',
 'find',
 'first',
 'for',
 'for the',
 'from',
 'from the',
 'fuck',
 'fuck o

In [148]:
y = np.array(df.label)
Counter(y)

Counter({'nonhostile': 3186, 'hostile': 3588})

In [149]:
y[[0,5,12]]

array(['nonhostile', 'hostile', 'hostile'], dtype=object)

In [150]:
class_names = set(df.label)
for word, idx in vec.vocabulary_.items():
    for class_name in class_names:
        class_idx = np.where(y==class_name)[0]
        print('%20s\t%20s\t%d' % (word, class_name, X[class_idx, idx].sum()))

               nigga	             hostile	922
               nigga	          nonhostile	1418
                 one	             hostile	119
                 one	          nonhostile	130
                 you	             hostile	1690
                 you	          nonhostile	932
                  me	             hostile	290
                  me	          nonhostile	322
                  go	             hostile	137
                  go	          nonhostile	85
                  to	             hostile	1030
                  to	          nonhostile	888
                hell	             hostile	38
                hell	          nonhostile	29
                 ass	             hostile	467
                 ass	          nonhostile	207
                that	             hostile	819
                that	          nonhostile	822
                done	             hostile	24
                done	          nonhostile	26
                 his	             hostile	200
                 his	          nonho

                  co	             hostile	186
                  co	          nonhostile	172
            https co	             hostile	184
            https co	          nonhostile	171
                 nah	             hostile	24
                 nah	          nonhostile	34
                when	             hostile	129
                when	          nonhostile	130
               think	             hostile	88
               think	          nonhostile	85
                 has	             hostile	102
                 has	          nonhostile	82
                been	             hostile	68
                been	          nonhostile	86
               doesn	             hostile	46
               doesn	          nonhostile	34
               every	             hostile	46
               every	          nonhostile	36
                time	             hostile	97
                time	          nonhostile	101
                into	             hostile	43
                into	          nonhostile	28
  

                very	          nonhostile	20
                 say	             hostile	94
                 say	          nonhostile	91
                some	             hostile	118
                some	          nonhostile	110
                made	             hostile	31
                made	          nonhostile	33
                love	             hostile	44
                love	          nonhostile	85
              before	             hostile	40
              before	          nonhostile	37
                yeah	             hostile	60
                yeah	          nonhostile	59
                rape	             hostile	22
                rape	          nonhostile	36
                  an	             hostile	128
                  an	          nonhostile	97
                then	             hostile	88
                then	          nonhostile	91
            you know	             hostile	29
            you know	          nonhostile	33
                long	             hostile	21
       

               pussy	          nonhostile	19
            fuck you	             hostile	76
            fuck you	          nonhostile	10
               lying	             hostile	29
               lying	          nonhostile	6
                word	             hostile	21
                word	          nonhostile	15
             they re	             hostile	23
             they re	          nonhostile	24
              it was	             hostile	29
              it was	          nonhostile	33
              at the	             hostile	26
              at the	          nonhostile	29
               great	             hostile	25
               great	          nonhostile	23
             cripple	             hostile	8
             cripple	          nonhostile	27
           nigga was	             hostile	20
           nigga was	          nonhostile	35
                 yes	             hostile	46
                 yes	          nonhostile	39
                 our	             hostile	33
            

In [151]:
clf = LogisticRegression(multi_class='auto')
clf.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [152]:
clf.coef_

array([[-1.42367304e-01,  1.57571422e-01,  2.55208666e-01,
        -2.03244629e-01, -7.78099380e-03, -9.31580046e-02,
         2.01694573e-01,  6.37632756e-01, -4.92342235e-01,
         1.47611601e-01,  2.10000319e-01,  2.74744231e-01,
         1.00371313e-01, -5.82958933e-02,  5.09999446e-01,
        -1.17167534e-02, -7.93861315e-02, -2.69487304e-01,
         6.34968710e-02, -4.32201540e-01, -2.52636050e-01,
         7.53765245e-02, -1.63747148e-01,  9.88122425e-02,
         2.88868799e-02,  5.39814211e-01, -1.06736964e+00,
        -7.11777822e-01,  6.32500850e-02,  3.88715300e-01,
         1.80589701e-01,  2.71355114e-01,  2.34421801e-01,
         2.93069602e-01,  5.23059934e-01,  6.51620729e-02,
        -3.83819188e-02,  6.07074015e-02,  6.09376224e-01,
        -6.21820723e-02,  3.39735060e-01,  4.98058541e-01,
        -8.06135015e-01, -8.28551294e-01, -2.03656140e-01,
        -5.54137260e-01,  2.49209091e-01,  6.16468945e-01,
        -4.93530719e-01, -3.17870861e-01,  3.99252140e-0

In [153]:
coef = [-clf.coef_[0], clf.coef_[0]]
print(coef)

[array([ 1.42367304e-01, -1.57571422e-01, -2.55208666e-01,  2.03244629e-01,
        7.78099380e-03,  9.31580046e-02, -2.01694573e-01, -6.37632756e-01,
        4.92342235e-01, -1.47611601e-01, -2.10000319e-01, -2.74744231e-01,
       -1.00371313e-01,  5.82958933e-02, -5.09999446e-01,  1.17167534e-02,
        7.93861315e-02,  2.69487304e-01, -6.34968710e-02,  4.32201540e-01,
        2.52636050e-01, -7.53765245e-02,  1.63747148e-01, -9.88122425e-02,
       -2.88868799e-02, -5.39814211e-01,  1.06736964e+00,  7.11777822e-01,
       -6.32500850e-02, -3.88715300e-01, -1.80589701e-01, -2.71355114e-01,
       -2.34421801e-01, -2.93069602e-01, -5.23059934e-01, -6.51620729e-02,
        3.83819188e-02, -6.07074015e-02, -6.09376224e-01,  6.21820723e-02,
       -3.39735060e-01, -4.98058541e-01,  8.06135015e-01,  8.28551294e-01,
        2.03656140e-01,  5.54137260e-01, -2.49209091e-01, -6.16468945e-01,
        4.93530719e-01,  3.17870861e-01, -3.99252140e-01, -1.56875499e-01,
       -1.00573812e+00, 

In [154]:
clf.classes_

array(['hostile', 'nonhostile'], dtype=object)

In [155]:
for ci, class_name in enumerate(clf.classes_):
    print('coefficients for %s' % class_name)
    display(pd.DataFrame([coef[ci]], columns=vec.get_feature_names()))

coefficients for hostile


,about,actually,after,again,ain,all,all the,already,also,always,...,you,you are,you can,you don,you have,you know,you re,your,your ass,yourself
0,0.142367,-0.157571,-0.255209,0.203245,0.007781,0.093158,-0.201695,-0.637633,0.492342,-0.147612,...,0.247989,0.445111,-0.43247,0.126951,-0.615671,-0.405049,-0.279251,0.489423,-0.104745,-0.244697


coefficients for nonhostile


,about,actually,after,again,ain,all,all the,already,also,always,...,you,you are,you can,you don,you have,you know,you re,your,your ass,yourself
0,-0.142367,0.157571,0.255209,-0.203245,-0.007781,-0.093158,0.201695,0.637633,-0.492342,0.147612,...,-0.247989,-0.445111,0.43247,-0.126951,0.615671,0.405049,0.279251,-0.489423,0.104745,0.244697


In [156]:
features = vec.get_feature_names()
for ci, class_name in enumerate(clf.classes_):
    print('top features for class %s' % class_name)
    for fi in coef[ci].argsort()[::-1][:10]: # descending order.
        print('%20s\t%.2f' % (features[fi], coef[ci][fi]))

top features for class hostile
                twat	2.80
                cunt	2.60
     realdonaldtrump	2.39
              retard	2.36
                shut	2.02
              faggot	1.85
            fuck off	1.78
               pussy	1.73
            fuck you	1.28
                 cum	1.24
top features for class nonhostile
             need to	1.28
                holy	1.14
                 mad	1.03
             calling	1.01
            what the	1.00
                love	0.95
            the only	0.85
            going to	0.81
                same	0.77
               video	0.75


In [157]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []
for train, test in kf.split(X):
    clf.fit(X[train], y[train])
    pred = clf.predict(X[test])
    accuracies.append(accuracy_score(y[test], pred))
    
    
print('accuracy over all cross-validation folds: %s' % str(accuracies))
print('mean=%.2f std=%.2f' % (np.mean(accuracies), np.std(accuracies)))

accuracy over all cross-validation folds: [0.6981549815498155, 0.6952029520295203, 0.6929889298892989, 0.7062730627306273, 0.7104874446085672]
mean=0.70 std=0.01
